In this article i will show how to create access to jupyter notebook form your local computer.
Whenever you ran jupyter with command:
```python
jupyter notebook
```
it will starts its own server, and through it it will serve a pages, typically under the address: http://localhost:8888  under which it can be either viewed or edited in a local browser.

In one of my [previous post](/vnc_on_virtual_machine.html) i described a mean how to make a VNC connection to remote computer. But often there is no such possiblity, or it would be just more convenient to access remote jupyter sever in local browser. In this article I will show how to establish such connection.

First in command line make sure that you have jupyter notebook installed and that it is working properly:
```shell
ptwnuk@instance04:~$ jupyter notebook
[I 13:39:33.961 NotebookApp] [nb_conda_kernels] enabled, 0 kernels found
[I 13:39:33.967 NotebookApp] Writing notebook server cookie secret to /run/user/1001/jupyter/notebook_cookie_secret
[I 13:39:34.209 NotebookApp] JupyterLab extension loaded from /opt/anaconda3/lib/python3.7/site-packages/jupyterlab
[I 13:39:34.209 NotebookApp] JupyterLab application directory is /opt/anaconda3/share/jupyter/lab
[I 13:39:34.363 NotebookApp] [nb_conda] enabled
[I 13:39:34.405 NotebookApp] ✓ nbpresent HTML export ENABLED
[W 13:39:34.405 NotebookApp] ✗ nbpresent PDF export DISABLED: No module named 'nbbrowserpdf'
[I 13:39:34.406 NotebookApp] Serving notebooks from local directory: /home/ptwnuk
[I 13:39:34.406 NotebookApp] The Jupyter Notebook is running at:
[I 13:39:34.406 NotebookApp] http://localhost:8888/?token=724c3c931cb0d68f46e24739c11d8bae0114a9c50ea3e5b3
[I 13:39:34.406 NotebookApp] Use Control-C to stop this server and shut down all kernels (twice to skip confirmatio
n).
[W 13:39:34.409 NotebookApp] No web browser found: could not locate runnable browser.
[C 13:39:34.410 NotebookApp] 
    
    Copy/paste this URL into your browser when you connect for the first time,
    to login with a token:
        http://localhost:8888/?token=724c3c931cb0d68f46e24739c11d8bae0114a9c50ea3e5b3
```

First we need to make sure we have a file jupyter_notebook_config.py with default settings for jupyter notebook.
You can chack that by executing:
```python
ls ~/.jupyter/jupyter_notebook_config.py
```
if you see the file, it means that you have configuration file created if not (there is error of no such file) you need to run the following command:
```python
jupyter notebook --generate-config
```

Now we need to modify jupyte default settings, by editing jupyter_notebook_config.py file by running:
 
```python
nano ~/.jupyter/jupyter_notebook_config.py
```

Inside that file the following lines should be added:
```shell
c = get_config()
c.NotebookApp.ip = '*' #Set the kernel’s IP address [default localhost]. 
#If the IP address is something other than localhost, 
#then Consoles on other machines will be able to connect to the Kernel
c.NotebookApp.port = 7000 #sets the port to 7000, to avoid conflicts with locally running jupyter
#c.NotebookApp.open_browser : False  Uncomment if you wish not to open remote browser for jupyter
```

Now you can run your server one more time, with the following parameters:
```python
jupyter-notebook --no-browser --port=7000
```
After that jupyter server is serving pages on 7000 port. 
In order to access jupyter notebook form remote computer you need to replace beginning of the address of server adress i.e *localhost* into IP of your remote computer. You can find IP of your machine in administrative console:
<img class="img-center" src="../../images/jupyter_vm/jupyter02.png" style="width: 400px"/>
Unfortunately, you would still not be able to access jupyter notebook. The reason is that by default all ports (except 22) are blocked by Google Cloud settings. So now we need to open out 7000 port for TCP protocols. In order to open this port we need to make a new custom firewall rule. You can do it by opening GCP main menu and then **Networking** and **Firewall rulles**:
<img class="img-center" src="../../images/jupyter_vm/fire01.png" style="width: 400px"/>

Here you can create a new firewall rule (port 7000, TCP, all instances in network, source IP: 0.0.0.0/0), it should look similar to the image below and then create button.
<img class="img-center" src="../../images/jupyter_vm/fire02.png" style="width: 400px"/>

After applying the firewall rule, you can one more time copy address provided by jupyter server, replace localhost adress by the external IP of your computer and paste that address into your browser:
<img class="img-center" src="../../images/jupyter_vm/jupyter03.png" style="width: 400px"/>

# Hints 
### Static IP
It is quite tedious to check IP address of your virtual machine. Especially each time you are starting a machine you usually have different external IP. In oder to avoid that complication you can reserve a static IP which can then be assign to any of your instances. You can get static IP in Menu/VPC Networ/External IP address. There you can click "Reserve static IP", which could be then attached to your instance. 

Just as a word of warning, if there is an static IP reserved which is not used (not assigned to any instances), you will be charged for it (0.01$ per hour). But any assigned IP to instances are free of charge, regardless if the instance is running or stopped. You can check status of your IP addresses if they are in sue by running following command **gcloud compute addresses list** in  Google Cloud Platform (command icon in top right corner of console).

### Easy logging to jupyter (without key)
The other inconvenience occurring with default configuration is that each time you start jupyter server you need to manually copy address from console and moreover you need to change the beginning address from 127.0.0.1 to external IP address. Those problems could be easily solved by modifying logging method from key to password. 
To change the logging method, after running server (jupyter-notebook --no-browser --port=7000) in your browser you need to provide jupyter server address **but without key**, like http://35.204.55.33:7000. Then you will see a welcome page, with a section: Setup a Password (like in the image below).
<img class="img-center" src="../../images/jupyter_vm/jupyter04.png" style="width: 400px"/>
There in the box Token paste a key (letters and numbers), which were provided during jupyter server initialization.
Additionally provide your own password in the "New Password" box, and then click: Log in and set new password.
This way you will establish your own password, which will have to be provided each time you will be logging to jupyter notebook through browser. 

Since now IP is static, and in address does not have any unique key, you can bookmark your jupyter server address (like: http://35.204.55.33:7000), and the access it with just single click. This makes a work flow with remote jupyter much smoother and enjoyable.